# External QoiModel tutorial

In this tutorial you will learn the basics of the QoiModel interface. The QoIModel interface is almost identical to the model interface,
but has an additional method called compute_qoi() where it returns a (usually scalar) quantity of interest. This type of model is useful for various workflows, such as training a ROM to a desired tolerance on a specified QoI. The API for the model interface is provided here:

https://pressio.github.io/rom-tools-and-workflows/romtools/workflows/models.html


In [5]:
#First, let's import the relavant modules:
import romtools
import os
import numpy as np
from matplotlib import pyplot as plt
from romtools.workflows import models

In [6]:
# As a starting point, we will use the model created in the external_model tutorial
from ipynb.fs.full.external_model import adrExternalRomToolsModel

In [11]:
#Now, we will create a QoI model. As a QoI, we will take the gradient of the state at the right of the domain
class adrExternalRomToolsQoiModel(adrExternalRomToolsModel):
        # Our class will inherit all the methods from adrExternalRomToolsModel
    
    def compute_qoi(self, run_directory: str, parameter_sample: dict):
        # Note that compute_qoi is always called after run_model
        solution = np.load(run_directory + '/solution.npz')
        u = solution['u']
        x = solution['x']
        dx = x[1] - x[0] #we use a uniform grid
        ux_at_right_edge = (0. - u[-1])/dx
        return ux_at_right_edge

#That's it!

In [12]:
#Let's do an example run of the model similar to how it will be used in a workflow.

if __name__=="__main__":
    # Instantiate our model
    myModel = adrExternalRomToolsQoiModel()

    #First, let's make a dictionary that is a stand in for the parameter space
    parameter_sample = {}
    parameter_sample['c'] = 0.5
    parameter_sample['nu'] = 1e-1

    #Now, let's populate our run directory. In this example, we will just use the current run directory.
    #In a full workflow, romtools will manage creating directories
    myModel.populate_run_directory(os.getcwd(),parameter_sample)

    #Now, let's run the model
    run_dir = os.getcwd()

    myModel.run_model(run_dir,parameter_sample)

    qoi = myModel.compute_qoi(run_dir,parameter_sample)

    print('The QoI is ' + str(qoi))
    #We will finish by cleaning up the files we created
    os.system('rm params.dat')
    os.system('rm solution.npz')

[1.28125738e-14 5.94704074e-02 1.18467441e-01 1.76917136e-01
 2.34733971e-01 2.91819061e-01 3.48058072e-01 4.03318803e-01
 4.57448398e-01 5.10270118e-01 5.61579606e-01 6.11140577e-01
 6.58679825e-01 7.03881455e-01 7.46380215e-01 7.85753780e-01
 8.21513841e-01 8.53095786e-01 8.79846785e-01 9.01012003e-01
 9.15718660e-01 9.22957608e-01 9.21562017e-01 9.10182740e-01
 8.87259825e-01 8.50989580e-01 7.99286485e-01 7.29739155e-01
 6.39559430e-01 5.25523499e-01 3.83903828e-01 2.10390458e-01
 0.00000000e+00] [0.      0.03125 0.0625  0.09375 0.125   0.15625 0.1875  0.21875 0.25
 0.28125 0.3125  0.34375 0.375   0.40625 0.4375  0.46875 0.5     0.53125
 0.5625  0.59375 0.625   0.65625 0.6875  0.71875 0.75    0.78125 0.8125
 0.84375 0.875   0.90625 0.9375  0.96875 1.     ]
The QoI is -6.7324946711637725
